In [44]:
import pandas as pd
import re

df = pd.read_excel("placas_kabum.xlsx")

print(df)

                                                  nome        valor
0    Placa de Vídeo RX 6600 CLD 8G ASRock AMD Radeo...  R$ 1.599,90
1    Placa de Vídeo RX 7600 GAMING OC 8G AMD Radeon...  R$ 1.999,99
2    Placa de Vídeo RX 6750XT Gaming XFX Graphics C...  R$ 2.599,99
3    Placa de Vídeo RX 6650XT XFX Speedster SWFT210...  R$ 1.879,99
4    Placa de Video RTX 4060 Infinity 2 Palit NVIDI...  R$ 2.299,99
..                                                 ...          ...
487  Placa de Video 1Gb GT710 Ddr3 64bit Vga/Dvi/Hd...    R$ 323,83
488  Placa de Vídeo AFOX GeForce GTX750TI, 2GB, GDD...  R$ 1.004,40
489  Placa de Vídeo Gainward NVIDIA GeForce GTX 166...  R$ 5.592,00
490  Adaptador Para Duas Fontes E Mineração Psuad 2...     R$ 21,90
491  Bloco VGA Barrow Para Rtx 3080 (ti) E Rtx 3090...    R$ 599,00

[492 rows x 2 columns]


In [45]:
# Converter valor para float
df["valor_float"] = (
    df["valor"]
    .str.replace("R\$", "", regex=True)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
C:\Users\47419992894\AppData\Local\Temp\ipykernel_12700\2785112618.py:4: SyntaxWarning: invalid escape sequence '\$'
  .str.replace("R\$", "", regex=True)


In [46]:

# Função para extrair GPU, marca e VRAM
def extrair_info(nome):
    nome_lower = nome.lower()
    gpu_match = re.search(r'(rtx|gtx|rx)\s?\d{3,4}(ti|super|xt)?', nome_lower)
    gpu = gpu_match.group(0).upper() if gpu_match else None

    marcas = ['ASUS', 'MSI', 'GIGABYTE', 'GALAX', 'XFX', 'ASROCK', 'ZOTAC', 'PALIT', 'PNY', 'ROG-STRIX', 
              'PCYES', 'SAPPHIRE', 'HWJ', 'INNO3D', 'VX PRO DUEX', 'PCWINMAX', 'PCI-E', 'COLORFUL', 'AFOX', 'REVENGER', 'POWERCOLOR', 'GAMER',
              'GAINWARD', 'BRX', 'TI EX WHITE', 'STAR', 'BLUECASE', 'VXPRO', 'PCIEXPRESS', 'DUEX', 'PCIE', 'SUPER AMP', 'KEEPDATA', 'AITEK', 'VX',
              'EX WHITE V2', 'VALIANTY', 'ARKTEK', 'XFX', 'KNUP', 'TGT', 'VX PRO', 'MANLI', 'EVOLUT', ]
    marca = next((m for m in marcas if m.lower() in nome_lower), None)

    vram_match = re.search(r'(\d{1,2})\s?g[bB]', nome)
    vram = int(vram_match.group(1)) if vram_match else None

    return pd.Series([gpu, marca, vram])

In [47]:
# Extraindo informações
df[["gpu", "marca", "vram_gb"]] = df["nome"].apply(extrair_info)

In [48]:
# Pontuações 
pontuacoes_gpu = {
    "GTX 1650": 50, "GTX 1660": 60, "GTX 1660TI": 65,
    "RTX 2060": 70, "RTX 3060": 80, "RTX 3060TI": 85, "RTX 3070": 90, "RTX 4060": 95, "RTX 4070": 100,
    "RX 6600": 75, "RX 6650XT": 78, "RX 6700XT": 88, "RX 6750XT": 90, "RX 7600": 92
}

In [49]:
# Colunas com cada pontuação das placas de vídeo
df["pontuacao_gpu"] = df["gpu"].map(pontuacoes_gpu).fillna(60)
df["pontuacao_vram"] = df["vram_gb"].fillna(6) * 10
df["pontuacao_total"] = df["pontuacao_gpu"] + df["pontuacao_vram"]
df["custo_beneficio"] = df["pontuacao_total"] / df["valor_float"]

In [50]:
# Tabela de GPU
placas_validas = df[df["gpu"].notnull()]

In [51]:
# Ranking
ranking = placas_validas.sort_values(by="custo_beneficio", ascending=False)
print(ranking[["nome", "gpu", "marca", "vram_gb", "valor_float", "pontuacao_total", "custo_beneficio"]].head(10))

df.to_excel('placas_kabum_tratado_3.xlsx', index=False)

                                                  nome         gpu  \
223  Placa de Video 1Gb GTX550TI Ddr5 128bit Vga/Dv...    GTX550TI   
323  Placa de Vídeo Afox NVIDIA GeForce GTX750TI, 2...    GTX750TI   
88   Placa De Vídeo Nvidia Geforce 4gb Gtx 550ti, G...   GTX 550TI   
256  Placa De Vídeo Afox, Nvidia Geforce GTX 750 4G...     GTX 750   
235  Placa De Vídeo BRX Geforce GTX 750, 4GB, 128 Bits     GTX 750   
115  Placa de Vídeo PowerColor AMD Radeon RX550, 4G...       RX550   
389  Bloco Vga Barrow Para Rtx 3060ti E 3070 Asus S...  RTX 3060TI   
201           Placa De Video Rx550 Aitek - Delta-rx550       RX550   
116  Placa De Video Gamer Nvidia Geforce Gtx750 GDD...      GTX750   
87   Placa de Vídeo Afox RX 550 AMD Radeon 4GB, GDD...      RX 550   

          marca  vram_gb  valor_float  pontuacao_total  custo_beneficio  
223        None      NaN       274.01            120.0         0.437940  
323        AFOX      NaN       382.23            120.0         0.313947  
88     